Este jupyter notebook provee una explicación respecto al análisis de tweets, considerando las distintas asunciones y justificaciones pertinentes en el proceso de tratamiento de datos; presentando las soluciones para los dos ejercicios propuestos en el el Modulo: <i>"Tratamiento de Datos"</i>, unidad 9 -> "Evaluación Final del Módulo"

# Ejercicio 1

<i>Dado una archivo que contiene en cada línea una palabra o conjunto de palabras seguido de un valor numérico, denominado “sentimiento”, y un conjunto de tweets, se pide calcular para cada tweet un valor denominado “sentimiento del tweet”, que se obtiene como la suma de los “sentimientos” de los términos que aparecen en el tweet.</i>

#### 1. Importación de las librerías para nuestro análisis

In [53]:
import json #lib. para leer la información de archivos con formato json
import html #lib. que será utilizada para convertir texto en formato HTML a string
from langdetect import detect #lib. para detectar el lenguaje de cada tweet

#### 2. Función para leer el archivo Sentimientos.txt

In [54]:
def read_Sentiments(file_sentimientos):
    sentiments = open(file_sentimientos)
    values={}
    for line in sentiments:
        term, value = line.split("\t")
        values[term] = int(value)
    return values

Leemos la lista de palabras y valores de Sentimientos.txt y lo guardamos en un diccionario llamado <i>"values"</i>

In [55]:
values = read_Sentiments("Sentimientos.txt") #read list of words and values from Sentimientos.txt

Se asume que el archivo "Sentimiento.txt" se encuentra en el mismo directorio que este notebook

#### 3. Inicializacion de variables

Inicializamos listas y diccionarios que nos permitirán guardar la información de los tweets. Se considera una 'mezcla' entre diccionarios y listas dado que ambas estructuras de datos en Python son mutables y nos permiten acceder a la información/elementos de manera ordenada.
Adicionalmente, se considera que los tweets estan en distintos idiomas, por lo que se crean distintos diccionarios para guardar la información.

In [6]:
tweets_arr = [] #create an empty list of tweets that will contain info in json format
tweets = {} #initialize empty tweets dictionary
english_tweets = {} #initialize an empty dict for tweets that are in English
other_tweets = {} #empty dictionary for tweets NOT in English
error_tweets = {} #empty dictionary for tweets that are either empty or are not words
final_values = [] #empty list that will store tuples (tweet, sentiment_value)

#### 4. Lectura del archivo Tweets.txt

Leemos el documento Tweets.txt y procedemos a guardar todas las cadenas JSON en la lista 'tweets_arr'. Posteriormente, se analizan los elementos en 'tweets_arr' y se guardan los tweets en el diccionario "tweets".

In [56]:
with open("Tweets.txt") as f: 
    for i in f: #for every line in the tweets.txt file append it to the tweets list
        tweets_arr.append(json.loads(i))

counter = 0 #initialize counter -> for indexing in the tweets dict
for j in range(len(tweets_arr)): #for every line stored in the tweets list
    if len(tweets_arr[j].keys()) != 1: #if the json dictionary has only one key
                                        #we assume it is NOT a 'tweet object'
        tweets[counter] = tweets_arr[j]["text"] #we save only the info from the 'text' key
        counter += 1

In [57]:
#descomentar las siguientes lineas para mostrar información inicial
#print("El número total de tweets registrados es de: "+str(counter))
#print("Los Tweets son:\n")
#tweets

#### 5. Validación del idioma en cada tweet

Ahora procedemos a analizar cada tweet, y determinar si está escrito en inglés y se lo guarda en el diccionario "english_tweets". En caso de que el tweet NO esté en inglés, lo guardamos en el diccionario "other_tweets", dado que pueden haber casos en los que <b>langdetect</b> presente falsos negativos.

In [58]:
en_count = 0
other_count = 0
err_count = 0
for k in range(len(tweets)): #Now we are going to analyse every tweet's text
    #first we filter by language -> if the text is in english, then we can calculate sentiment
    #if the text is NOT in English we can just set the sentiment value to 0    
    texto = tweets[k]
    try:
        if(detect(texto.lower())=='en'): #if the tweet is in Enlgish
            english_tweets[en_count] = html.unescape(texto) #save the unescaped text in English
            en_count += 1

        else:
            other_tweets[other_count] = html.unescape(texto) #save the unescaped non-english text in other tweets
            other_count += 1
        
    except:
        error_tweets[err_count] = html.unescape(texto) #save the unescaped error text in error_tweets
        err_count += 1

#### 6. Cálculo del sentimiento

Se define la función <b><i>calc_sent</i></b>, la cual determinará el valor/sentimiento del tweet tras realizar una búsqueda de palabras en el diccionario "values"

In [59]:
def calc_sent(text):
    sentiment_value = 0 #initialize sentiment value as 0
    #text is the string that corresponds to the tweet
    text = text.split(" ") #we split the string; regex:whitespace
    #now we check each word against the values dictionary
    for word in text:
        try:
            sentiment_value += values[word.lower()]
            #read something from the values
        except:
            #print("the word is not in the list")
            sentiment_value += 0
    
    #we save the results in a tuple that will then be stored in: final_values
    return(sentiment_value)

Procesamos los tweets en inglés (english_tweets), y los tweets que están en otro idioma (other tweets).
Para el caso de <i>'other tweets'</i> solamente incluiremos en el listado final (final_values), aquellos tweets con un sentimiento diferente de 0.


In [67]:
#Process English tweets
#print("\nAnalyzing Tweets in Enlgish...\n")
for tweet in english_tweets:
    v = calc_sent(english_tweets[tweet])
    temp = (english_tweets[tweet],v)
    final_values.append(temp)

#Processing tweets that were not considered English text
#print("\nAnalyzing Tweets in other languages...\n")
for tweet in other_tweets:
    v=calc_sent(other_tweets[tweet])
    temp = (other_tweets[tweet],v)
    if(v!=0):
        final_values.append(temp)

#### 7. Resultado final

Una vez que tenemos lista final de tweets en "final_values" procedemos a imprimir los resultados para cada tweet, siguiéndo el formato sugerido en el aula virtual.

In [71]:
print("El número total de tweets analizados es de: "+str(len(final_values)))

El número total de tweets analizados es de: 991


In [69]:
#=====================FINAL OUTPUT FOR EXERCISE 1==============================
for item in final_values:
    print("EL SIGUIENTE TWEET: '"+item[0]+"' TIENE UN SENTIMIENTO ASOCIADO DE: "+str(item[1])+"\n")

EL SIGUIENTE TWEET: '@Brenamae_ I WHALE SLAP YOUR FIN AND TELL YOU ONE LAST TIME: GO AWHALE' TIENE UN SENTIMIENTO ASOCIADO DE: 0

EL SIGUIENTE TWEET: 'RT @byunghns: 😭 I LOVE #틴탑 SO MUCH #쉽지않아 IS GOING TO BE SO GOOD 😭' TIENE UN SENTIMIENTO ASOCIADO DE: 6

EL SIGUIENTE TWEET: 'RT @baileymac02: CAN'T WAIT! I am playing @SoccerSix at The London Soccerdome on SUNDAY 26TH OCTOBER! Tickets selling fast: http://t.co/c6i…' TIENE UN SENTIMIENTO ASOCIADO DE: 0

EL SIGUIENTE TWEET: 'RT @Nashgrier: Happy Birthday to one of my best friends @jackgilinsky Can't wait till you're out in LA! Have a good one!' TIENE UN SENTIMIENTO ASOCIADO DE: 9

EL SIGUIENTE TWEET: 'I liked a @YouTube video from @jordzcreations http://t.co/YJhzCimXHG TOP 5 RATED PLAYERS IN FIFA15! (MESSI, RONALDO & IBRA!)' TIENE UN SENTIMIENTO ASOCIADO DE: 4

EL SIGUIENTE TWEET: 'RT @KirkJTorrance: Cat's out of the bag! "@WilliamJHague: giving Scotland more powers if it votes No is not Government policy" http://t.co/…' TIENE UN SENTIMIEN

# Ejercicio 2

<i>"Dado un archivo que contiene en cada línea una palabra o conjunto de palabras seguido de un valor numérico denominado “sentimiento” y un conjunto de tweets, se pide calcular el sentimiento de aquellas palabras o conjunto de palabras que no tienen un valor asociado en el archivo de “sentimientos”. Se pueden seguir distintas estrategias para asignar un valor. Por ejemplo, se podría asignar como valor el valor del “sentimiento” del tweet en que se encuentra la palabra o conjunto de palabras sin valor, o el valor medio del “sentimiento” del tweet."</i>

Una vez que hemos calculado el sentimiento de los tweets (resultado final del ejercicio 1), consideramos aquellas palabras que constan en el tweet y que NO tienen un valor asignado.
El proceso para este análisis será (a grosso modo) el siguiente:
<ol>
  <li>Leer el tweet evaluado en el ejercicio 1 </li>
  <li>Fragmentamos el tweet usando el método String.split()</li>
  <li>Depurar / Limpiar el contenido del tweet</li>
  <li>Asignar valor de 0 a las palabras que se encuentren en el set Stop-Words (stpwrds)</li>
  <li>Para otras palabras, revisar si se encuentran en el diccionario 'values', caso contrario asignarles el valor promedio del sentimiento del tweet.</li>
</ol>

#### 1. Importación de librerías o módulos para el análisis

Se utilizará NLTK (package empleado en PLN), esto nos permitirá 'limpiar' las palabras de los tweets y 'tokenizarlas' para nuestro análisis.

In [2]:
!pip install nltk
#nltk.download(all)
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

Ahora añadimos ciertas palabras al set de stopwords. En este caso, se asume que RT y rt no aportan valor al tweet.

In [72]:
stpwrds = set(stopwords.words('english'))
stpwrds.add('RT')
stpwrds.add('rt')
stpwrds.add("n't")

#### 2. Definición de funciones para análisis de palabras y texto

Definimos la función <b><i>cleanWord</i></b> la cual depura una palabra que contenga signos de puntuación, emojis, etc.

In [73]:
def cleanWord(word):
    word = word_tokenize(word.lower()) #we tokenize the 'word'
    for w in word: #we try to depur the caracters or substrings that are NOT alphanumeric
        if(w.isalnum()==False and len(w)<=1): #for example when we have hypened words (i.e. pinkish-blue.)
            word.remove(w) #remove substrings or characters that are NOT alpahumeric and may cause confusion         
    return word

Definimos la función <b><i>joiner</i></b>, la cual asume una lista de 2 o mas sub-strings que deben ser unidas

In [15]:
def joiner(element): #we assume element is a list of 2 or more strings (i.e. ["should","n't"])
    temp=""
    for e in element:
        temp += e #we concatenate strings as they are at that moment
    return temp

#### 3. Análisis de los tweets por medio de la función <i>wordValues</i>

Definimos la función <b><i>wordValues</i></b>, la cual considera como entrada el tweet (string) y su valor asociado (int), descompone el texto en palabras y se realiza un análisis de los elementos que conforman el tweet.

In [76]:
def wordValues(tweet, value): # tweet-> String, value -> integer
    #we check the number of alphanumeric subelements
    sub_elements=[] #this list will contain a tuple of (element,mean-value)
    flags=[] #list of elements and flags (multiplication factor for mean-value)
    num_alnum = 0 #number of ~alphanumeric elements
    splitted = tweet.split(" ") #we save the splitted text in the variable
        
    for element in splitted: #for every element in the splitted tweet
        
        #we first do a clean-up of the word
        if(len(cleanWord(element))>1):
            temp_element = joiner(cleanWord(element))
        else:
            if(len(cleanWord(element))==0): #if the element is an emoji or a non-alphanumeric, cleanWorld will return an empty list
                temp_element = "" #we then assign an empty string to it
            else:
                temp_element = cleanWord(element)[0] #we access the 0th element because cleanWord returns a list -> from tokenization
        
        #we can now validate: 1) if the element is in the Stop-Words set or 2) if it contains 'useless' info
                
        if(temp_element.isalpha()):
            try: #now we check if the word/element is a key from the values dictionary
                if(values[element.lower()]!=0):
                    num_alnum += 1
                    pass #this would be the case for a word that was used to calculate the first value of sentiment for the tweet
                
            except: #if the element contains some punctuation for example
                try:
                    if(values[temp_element]!=0):
                        flags.append((element,values[temp_element])) #append the element to the flags list and do not consider it for the mean-value calculation
                except:
                    if(temp_element not in stpwrds):
                        if((element.startswith("@") or ("@" in element)) or element.startswith("http")): #the http condition is just in case; however it is not likely to ever happen
                            flags.append((element,0))
                        else:
                            flags.append((element,1))
                            num_alnum += 1
                    else:
                        flags.append((element,0))
                    
        else:
            partial = "".join(filter(str.isalpha,element))
            if((partial.isalpha()) and (partial[:4]!="http") and ("@" not in element)): #if the element is not exactly alpahumeric but has some sense of it then consider it alphanumeric
                flags.append((element,1))
                num_alnum += 1
            else:
                flags.append((element,0))

    if (num_alnum == 0): #if there are no alphanumeric elements in the tweet whatsoever
        mean_value = 0.00
    else:
        mean_value = round(value/num_alnum,2) #if the value is 0 then the mean value will be 0 too
    
    for flag_element in flags: #now for every tuple in the flags list
        if(flag_element[1]!=(0 or 1)):
            sub_elements.append((flag_element[0],flag_element[1]))
        else:
            sub_elements.append((flag_element[0],(flag_element[1]*mean_value)))
        
    return [sub_elements, num_alnum]        

Se define la función <b><i>displayValues</i></b>, para desplegar la información de cada palabra dentro del tweet

In [88]:
def displayValues(tweet,sentimentValue):
    [example,n_div]=wordValues(tweet,sentimentValue)
    print("\n====================================================================================")
    print("El tweet que será analizado es: \n'"+tweet+"'")
    print("El valor asociado a este tweet fue: "+str(sentimentValue))
    print("-------------------------------------------------------------------------------------")
    print("El valor promedio se obtuvo de dividir: "+ str(sentimentValue) +" entre "+str(n_div))
    print("Los resultados son los siguientes: \n")

    for line in example:    
            print("%s: %.2f" % (line[0], line[1])) #formatting to round the value to 2 decimal places

#### 4. Resultado final

Se procede a analizar todos los tweets dentro de <i>"final values"</i>

<i>"Como resultado se debe mostrar por pantalla en cada línea, una palabra o conjunto de palabras y a continuación un valor numérico que represente el “sentimiento” de dicha palabra o conjunto de palabras. Solo se mostrarán la palabra o conjunto de palabras que no aparecían con un valor en el archivo original."</i>

In [89]:
#------------------------------------------------------------------------------
for i in range(len(final_values)):
    #print("Analysis for Tweet: '"+final_values[i][0]+"' \n")
    tweet = final_values[i][0]
    sentiment = final_values[i][1]
    displayValues(tweet,sentiment)


El tweet que será analizado es: 
'@Brenamae_ I WHALE SLAP YOUR FIN AND TELL YOU ONE LAST TIME: GO AWHALE'
El valor asociado a este tweet fue: 0
-------------------------------------------------------------------------------------
El valor promedio se obtuvo de dividir: 0 entre 9
Los resultados son los siguientes: 

@Brenamae_: 0.00
I: 0.00
WHALE: 0.00
SLAP: 0.00
YOUR: 0.00
FIN: 0.00
AND: 0.00
TELL: 0.00
YOU: 0.00
ONE: 0.00
LAST: 0.00
TIME:: 0.00
GO: 0.00
AWHALE: 0.00

El tweet que será analizado es: 
'RT @byunghns: 😭 I LOVE #틴탑 SO MUCH #쉽지않아 IS GOING TO BE SO GOOD 😭'
El valor asociado a este tweet fue: 6
-------------------------------------------------------------------------------------
El valor promedio se obtuvo de dividir: 6 entre 6
Los resultados son los siguientes: 

RT: 0.00
@byunghns:: 0.00
😭: 0.00
I: 0.00
#틴탑: 1.00
SO: 0.00
MUCH: 1.00
#쉽지않아: 1.00
IS: 0.00
GOING: 1.00
TO: 0.00
BE: 0.00
SO: 0.00
😭: 0.00

El tweet que será analizado es: 
'RT @baileymac02: CAN'T WAIT! I am playi

El tweet que será analizado es: 
'RT @aartic02: Victory of AAP will be Victory of Common People #OnlyAAP4Delhi'
El valor asociado a este tweet fue: 0
-------------------------------------------------------------------------------------
El valor promedio se obtuvo de dividir: 0 entre 6
Los resultados son los siguientes: 

RT: 0.00
@aartic02:: 0.00
Victory: 0.00
of: 0.00
AAP: 0.00
will: 0.00
be: 0.00
Victory: 0.00
of: 0.00
Common: 0.00
People: 0.00
#OnlyAAP4Delhi: 0.00

El tweet que será analizado es: 
'Why waste you time on someone when they don't give you any of their time'
El valor asociado a este tweet fue: -1
-------------------------------------------------------------------------------------
El valor promedio se obtuvo de dividir: -1 entre 6
Los resultados son los siguientes: 

Why: 0.00
you: 0.00
time: -0.17
on: 0.00
someone: -0.17
when: 0.00
they: 0.00
don't: -0.17
give: -0.17
you: 0.00
any: 0.00
of: 0.00
their: 0.00
time: -0.17

El tweet que será analizado es: 
'RT @PBirdly: Ju


El tweet que será analizado es: 
'RT @PoetryPix: Retweet this tweet for a chance to win a small Lokai Bracelet. MUST BE FOLLOWING. Ends September 24th. http://t.co/C1lrUAsFlg'
El valor asociado a este tweet fue: 6
-------------------------------------------------------------------------------------
El valor promedio se obtuvo de dividir: 6 entre 12
Los resultados son los siguientes: 

RT: 0.00
@PoetryPix:: 0.00
Retweet: 0.50
this: 0.00
tweet: 0.50
for: 0.00
a: 0.00
to: 0.00
a: 0.00
small: 0.50
Lokai: 0.50
Bracelet.: 0.50
MUST: 0.50
BE: 0.00
FOLLOWING.: 0.50
Ends: 0.50
September: 0.50
24th.: 0.50
http://t.co/C1lrUAsFlg: 0.00

El tweet que será analizado es: 
'RT @C_Rivers24: @Calvinminds11 I'm honestly just chilling dude 😌'
El valor asociado a este tweet fue: -1
-------------------------------------------------------------------------------------
El valor promedio se obtuvo de dividir: -1 entre 4
Los resultados son los siguientes: 

RT: 0.00
@C_Rivers24:: 0.00
@Calvinminds11: 0.00
I'm:

ring!: 0.00
http://t.co/MaGkcNtzkI: 0.00

El tweet que será analizado es: 
'I was busy being naked in the forest!!! RT @FieryDaemon: You haven't spoken to your brother yet??? The hell is wrong with you???'
El valor asociado a este tweet fue: -6
-------------------------------------------------------------------------------------
El valor promedio se obtuvo de dividir: -6 entre 10
Los resultados son los siguientes: 

I: 0.00
busy: -0.60
being: 0.00
naked: -0.60
in: 0.00
the: 0.00
forest!!!: -0.60
RT: 0.00
@FieryDaemon:: 0.00
You: 0.00
haven't: -0.60
spoken: -0.60
to: 0.00
your: 0.00
brother: -0.60
yet???: -0.60
The: 0.00
is: 0.00
with: 0.00
you???: -0.60

El tweet que será analizado es: 
'Real shit http://t.co/3sQK6ILWq7'
El valor asociado a este tweet fue: -4
-------------------------------------------------------------------------------------
El valor promedio se obtuvo de dividir: -4 entre 2
Los resultados son los siguientes: 

Real: -2.00
http://t.co/3sQK6ILWq7: 0.00

El tweet que s


El tweet que será analizado es: 
'RT @Nashgrier: Happy birthday @tfionash I love you soooo much! Thank you for everything, you have no idea how much you mean to me. Hope it'…'
El valor asociado a este tweet fue: 9
-------------------------------------------------------------------------------------
El valor promedio se obtuvo de dividir: 9 entre 13
Los resultados son los siguientes: 

RT: 0.00
@Nashgrier:: 0.00
birthday: 0.69
@tfionash: 0.00
I: 0.00
you: 0.00
soooo: 0.69
much!: 0.69
you: 0.00
for: 0.00
everything,: 0.69
you: 0.00
have: 0.00
idea: 0.69
how: 0.00
much: 0.69
you: 0.00
mean: 0.69
to: 0.00
me.: 0.00
it'…: 0.69

El tweet que será analizado es: 
'RT @eglobalinfo: eGlobal Solutions | Coming Soon http://t.co/c6nGXQlVI1'
El valor asociado a este tweet fue: 1
-------------------------------------------------------------------------------------
El valor promedio se obtuvo de dividir: 1 entre 4
Los resultados son los siguientes: 

RT: 0.00
@eglobalinfo:: 0.00
eGlobal: 0.25
|: 0.00

...

Ewig: 0.00
#on: 0.00
Schlagergarage: 0.00

El tweet que será analizado es: 
'I just took a test where the answer to a question was Mozart's dad.'
El valor asociado a este tweet fue: 0
-------------------------------------------------------------------------------------
El valor promedio se obtuvo de dividir: 0 entre 6
Los resultados son los siguientes: 

I: 0.00
just: 0.00
took: 0.00
a: 0.00
test: 0.00
where: 0.00
the: 0.00
answer: 0.00
to: 0.00
a: 0.00
question: 0.00
Mozart's: 0.00
dad.: 0.00

El tweet que será analizado es: 
'RT @Louis_Tomlinson: The support for the new album and Fireproof has been amazing ! So so excited for you all to hear more ! Loveeeee'
El valor asociado a este tweet fue: 9
-------------------------------------------------------------------------------------
El valor promedio se obtuvo de dividir: 9 entre 8
Los resultados son los siguientes: 

RT: 0.00
@Louis_Tomlinson:: 0.00
The: 0.00
for: 0.00
the: 0.00
new: 1.12
album: 1.12
and: 0.00
Fireproof: 1.12
has: 0.00


El tweet que será analizado es: 
'RT @PoetryPix: Retweet this tweet for a chance to win a small Lokai Bracelet. MUST BE FOLLOWING. Ends September 24th. http://t.co/C1lrUAsFlg'
El valor asociado a este tweet fue: 6
-------------------------------------------------------------------------------------
El valor promedio se obtuvo de dividir: 6 entre 12
Los resultados son los siguientes: 

RT: 0.00
@PoetryPix:: 0.00
Retweet: 0.50
this: 0.00
tweet: 0.50
for: 0.00
a: 0.00
to: 0.00
a: 0.00
small: 0.50
Lokai: 0.50
Bracelet.: 0.50
MUST: 0.50
BE: 0.00
FOLLOWING.: 0.50
Ends: 0.50
September: 0.50
24th.: 0.50
http://t.co/C1lrUAsFlg: 0.00

El tweet que será analizado es: 
'RT @C_Rivers24: @Calvinminds11 I'm honestly just chilling dude 😌'
El valor asociado a este tweet fue: -1
-------------------------------------------------------------------------------------
El valor promedio se obtuvo de dividir: -1 entre 4
Los resultados son los siguientes: 

RT: 0.00
@C_Rivers24:: 0.00
@Calvinminds11: 0.00
I'm:


El tweet que será analizado es: 
'RT @RottenTomatoes: New pictures from #OnceUponATime season 4 ----> http://t.co/UOdg3NDnAh #OUAT @OnceABC http://t.co/yDN1i0cvwi'
El valor asociado a este tweet fue: 0
-------------------------------------------------------------------------------------
El valor promedio se obtuvo de dividir: 0 entre 5
Los resultados son los siguientes: 

RT: 0.00
@RottenTomatoes:: 0.00
New: 0.00
pictures: 0.00
from: 0.00
#OnceUponATime: 0.00
season: 0.00
4: 0.00
---->: 0.00
http://t.co/UOdg3NDnAh: 0.00
#OUAT: 0.00
@OnceABC: 0.00
http://t.co/yDN1i0cvwi: 0.00

El tweet que será analizado es: 
'RT @Nashgrier: New Skit Tomorrow? Subscribe! 😛 - http://t.co/kn58WSAPaL'
El valor asociado a este tweet fue: 0
-------------------------------------------------------------------------------------
El valor promedio se obtuvo de dividir: 0 entre 4
Los resultados son los siguientes: 

RT: 0.00
@Nashgrier:: 0.00
New: 0.00
Skit: 0.00
Tomorrow?: 0.00
Subscribe!: 0.00
😛: 0.00
-: 0.00
h

Los resultados son los siguientes: 

RT: 0.00
@BestofCancer:: 0.00
#Cancer: -1.00
needs: 0.00
sincerity.: 2.00

El tweet que será analizado es: 
'Just waking up in the morning gotta thank G O D ❤️❤️❤️❤️❤️🙏🙏'
El valor asociado a este tweet fue: 2
-------------------------------------------------------------------------------------
El valor promedio se obtuvo de dividir: 2 entre 5
Los resultados son los siguientes: 

Just: 0.00
waking: 0.40
up: 0.00
in: 0.00
the: 0.00
morning: 0.40
gotta: 0.40
G: 0.40
O: 0.00
D: 0.00
❤️❤️❤️❤️❤️🙏🙏: 0.00

El tweet que será analizado es: 
'FFC IP's Symbaloo - Ozbirn's Google Mix #FCSLearn #FFCSoar #alpartners @FFCSoar http://t.co/Ub2wkOWat8'
El valor asociado a este tweet fue: 0
-------------------------------------------------------------------------------------
El valor promedio se obtuvo de dividir: 0 entre 9
Los resultados son los siguientes: 

FFC: 0.00
IP's: 0.00
Symbaloo: 0.00
-: 0.00
Ozbirn's: 0.00
Google: 0.00
Mix: 0.00
#FCSLearn: 0.00
#FFCSoar: 0.


El tweet que será analizado es: 
'RT @ChochoMartinez1: Cami mientras me cagaba a pedo ,  la abrazaba y no la dejaba ir jajajajaja n.n'
El valor asociado a este tweet fue: -1
-------------------------------------------------------------------------------------
El valor promedio se obtuvo de dividir: -1 entre 12
Los resultados son los siguientes: 

RT: 0.00
@ChochoMartinez1:: 0.00
Cami: -0.08
mientras: -0.08
me: 0.00
cagaba: -0.08
a: 0.00
pedo: -0.08
,: 0.00
: 0.00
la: -0.08
abrazaba: -0.08
y: 0.00
la: -0.08
dejaba: -0.08
ir: -0.08
jajajajaja: -0.08
n.n: -0.08
